In [ ]:
# Para um arquivo mais limpo (sem dependências do sistema):
#conda env export --from-history > environment.yml

In [ ]:
class Cidade:
    def __init__(self, Nome, UF):
        self.Nome = Nome
        self.UF = UF
        self.arestas_saida = {}

    def __repr__(self):
        saidas = "\n"
        for cidade, (dist, usa_barco) in self.arestas_saida.items():
            saidas += (
                f"\t→ {cidade.UF:<3} | "
                f"Distancia: {dist:>5} | "
                f"Usa barco: {'Sim' if usa_barco else 'Não'}\n"
            )

        return (
            f"-- {self.Nome} ({self.UF}) --\n"
            f"Arestas de saída:{saidas}\n"
        )

    def __hash__(self):
        return hash(self.UF)

    def __eq__(self, outra):
        return isinstance(outra, Cidade) and self.UF == outra.UF
        

    def adicionar_aresta(self, destino, distancia, usa_barco):
        self.arestas_saida[destino] = (distancia, usa_barco)

In [ ]:
def carregarCSV(txt):
    with open(txt, 'r', encoding='utf-8') as arq:
        dados = arq.readlines()
    
    cidades = {}

    #Criar cidades
    for linha in dados:
        linha = linha.strip()
        aux = linha.split(",")

        nome = aux[0]
        uf = aux[1]

        cidades[uf] = Cidade(nome, uf)

    #Criar vizinhanças
    for linha in dados:
            linha = linha.strip()
            aux = linha.split(",")
    
            cidade_atual = cidades[aux[1]]
    
            for coluna in aux[2:]:
                usa_barco = False
    
                if coluna[0] == "*":
                    coluna = coluna[1:]
                    usa_barco = True
    
                destino_uf, dist = coluna.split(":")
                dist = int(dist)
    
                cidade_atual.adicionar_aresta(
                    cidades[destino_uf],
                    dist,
                    usa_barco
                )     
    return list(cidades.values())
    
dados = carregarCSV("../data/processed/cidades.txt")

In [ ]:
from math import inf

def dijkstra(cidades, origem, recebe_penalidade = False):
    """
    Calcula a menor distancia 
    dijkstra(cidades, origem)

    cidades -> Lista de objetos Cidade
    origem -> String referente ao UF da cidade. Ex.: "RJ", "SP", "RS", "PE"
    """
    # Dicinários de objetos cidade com os valores iniciais
    distancia = {cidade: inf for cidade in cidades}
    pai = {cidade: None for cidade in cidades}
    visitado = {cidade: False for cidade in cidades}

    distancia[origem] = 0

    while True:
        # Escolher a cidade não visitada com menor distância
        u = None
        menor_dist = inf

        for cidade in cidades:
            if not visitado[cidade] and distancia[cidade] < menor_dist:
                menor_dist = distancia[cidade]
                u = cidade

        # Se não achou nenhuma, terminou
        if u is None:
            break

        visitado[u] = True

        # Relaxamento das arestas
        for v, (peso, usa_barco) in u.arestas_saida.items():
            if not visitado[v]:
                peso = peso*1.5 if usa_barco and recebe_penalidade else peso
                nova_dist = distancia[u] + peso
                if nova_dist < distancia[v]:
                    distancia[v] = nova_dist
                    pai[v] = u

    return distancia, pai

In [ ]:
def construir_rota(pai, origem, destino):
    caminho = []
    atual = destino

    while atual is not None:
        caminho.append(atual)
        atual = pai[atual]

    caminho.reverse()

    if caminho[0] != origem:
        return None

    return caminho

In [ ]:
import geopandas as gpd

def carregar_mapa_brasil():
    return gpd.read_file(
        "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
    )


In [ ]:
def centroides_por_uf(mapa):
    coords = {}

    for _, row in mapa.iterrows():
        uf = row["sigla"]  # GARANTIDO
        centro = row.geometry.centroid
        coords[uf] = (centro.x, centro.y)

    return coords


In [ ]:
def desenhar_mapa_base(mapa):
    fig, ax = plt.subplots(figsize=(10, 10))

    mapa.plot(
        ax=ax,
        color="whitesmoke",
        edgecolor="black",
        linewidth=0.8
    )

    ax.set_title("Mapa do Brasil - Estados")
    ax.axis("off")

    return fig, ax


In [ ]:
def desenhar_caminho(ax, caminho, coords, recebe_penalidade=False):
    distancia_total = 0
    linhas_legenda = []

    # Pontos (estados visitados)
    for cidade in caminho:
        x, y = coords[cidade.UF]
        ax.plot(x, y, "o", color="blue", markersize=2)
        ax.text(x, y, cidade.UF, fontsize=15, ha="center", va="center")

    # Setas entre os estados
    for i in range(len(caminho) - 1):
        c1 = caminho[i]
        c2 = caminho[i + 1]

        x1, y1 = coords[c1.UF]
        x2, y2 = coords[c2.UF]

        # Info da aresta
        dist, usa_barco = c1.arestas_saida[c2]

        # Penalidade
        penalidade = 0.5 * dist if usa_barco and recebe_penalidade else 0
        dist_exibida = dist + penalidade
        distancia_total += dist_exibida

        # Cor e tipo
        cor = "blue" if usa_barco else "black"
        tipo = "Barco" if usa_barco else "Terra"

        # Desenhar seta
        ax.annotate(
            "",
            xy=(x2, y2),
            xytext=(x1, y1),
            arrowprops=dict(
                arrowstyle="->",
                color=cor,
                linewidth=1.5
            )
        )

        # Texto no mapa (meio da seta)
        xm = (x1 + x2) / 2
        ym = (y1 + y2) / 2

        texto_mapa = (
            f"{dist} + {int(penalidade)}"
            if usa_barco and recebe_penalidade else f"{dist}"
        )

        ax.text(
            xm,
            ym,
            texto_mapa,
            fontsize=10,
            color=cor,
            ha="center",
            va="center",
            bbox=dict(
                boxstyle="round,pad=0.2",
                fc="white",
                ec=cor,
                alpha=0.85
            )
        )

        # Linha da legenda
        linhas_legenda.append(
            f"{c1.UF} → {c2.UF} | {tipo} | {dist} + {int(penalidade)} = {int(dist_exibida)}"
            if usa_barco and recebe_penalidade
            else f"{c1.UF} → {c2.UF} | {tipo} | {dist}"
        )

    # ----- LEGENDA LATERAL -----
    #legenda_texto = "Com penalidades\n\n" if recebe_penalidade else "Sem penalidades\n\n"
    legenda_texto = "Rota por segmento\n\n"
    legenda_texto += "\n".join(linhas_legenda)
    legenda_texto += f"\n\nDistância total: {int(distancia_total)}"

    """ax.text(
        1.02, 0.5,
        legenda_texto,
        transform=ax.transAxes,
        fontsize=10,
        va="center",
        ha="left",
        bbox=dict(
            boxstyle="round,pad=0.5",
            fc="white",
            ec="black",
            alpha=0.9
        )
    )"""
    
    titulo = "Com penalidades" if recebe_penalidade else "Sem penalidades"

    ax.text(
        1.02, 0.95,
        titulo,
        transform=ax.transAxes,
        fontsize=12,
        fontweight="bold",
        va="top",
        ha="left"
    )

    ax.plot([1.02, 1.25], [0.93, 0.93], transform=ax.transAxes, color="black", lw=1)
    
    ax.text(
        1.02, 0.90,
        legenda_texto,
        transform=ax.transAxes,
        fontsize=10,
        va="top",
        ha="left"
    )



In [ ]:
origem = next(c for c in dados if c.UF == "AP")
destino = next(c for c in dados if c.UF == "ES")

usaPenalidade = True

dist, pai = dijkstra(dados, origem, recebe_penalidade = usaPenalidade)
caminho = construir_rota(pai, origem, destino)

print("Distância mínima:", dist[destino])
print("Caminho:")
for c in caminho:
    print(f"{c.Nome} ({c.UF})")


mapa = carregar_mapa_brasil()
coords = centroides_por_uf(mapa)
fig, ax = desenhar_mapa_base(mapa)
desenhar_caminho(ax, caminho, coords, recebe_penalidade = usaPenalidade)

plt.show()


In [ ]:
origem = next(c for c in dados if c.UF == "AP")
destino = next(c for c in dados if c.UF == "ES")

usaPenalidade = False

dist, pai = dijkstra(dados, origem, recebe_penalidade = usaPenalidade)
caminho = construir_rota(pai, origem, destino)

print("Distância mínima:", dist[destino], "km")
print("Caminho:")
prev = dist[origem]
for c in caminho:
    print(f"{c.Nome} ({c.UF}) => Trajeto atual: {dist[c]-prev} km | Acumulada: {dist[c]} km")
    prev = dist[c]


mapa = carregar_mapa_brasil()
coords = centroides_por_uf(mapa)
fig, ax = desenhar_mapa_base(mapa)
desenhar_caminho(ax, caminho, coords, recebe_penalidade = usaPenalidade)

plt.show()
